In [253]:
import requests 
from bs4 import BeautifulSoup 
from datetime import date, datetime
import pandas as pd
from openpyxl import load_workbook
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows

In [254]:
vr_snapshot_path = 'https://www.valueresearchonline.com/funds/newsnapshot.asp?schemecode='
vr_performance_path = 'https://www.valueresearchonline.com/funds/fundperformance.asp?schemecode='
vr_portfolio_path = 'https://www.valueresearchonline.com/funds/portfoliovr.asp?schemecode='
vr_analysis_path = 'https://www.valueresearchonline.com/funds/fundanalysis.asp?schemecode='

et_performance_path = 'https://economictimes.indiatimes.com/mfreturns/schemeid-'
et_portfolio_path = 'https://economictimes.indiatimes.com/mfportfolio/schemeid-'

details_file_path = 'C:/Prajeen/My Docs/Equity/1.Equity Details/1.Fund Analysis/Fund Details/'

masterList = '1.fund_master_list.xlsx'
#masterList = '3.Single_fund_list.xlsx'

isSingleFundType = True
singleFundType = 'large cap'

In [255]:
def readMasterExcel():
    master_wb = load_workbook(details_file_path+masterList)
    fundTypes = {}
    sheets = master_wb.get_sheet_names()
    for sheet in sheets:
        fundTypes[sheet] = master_wb[sheet]
    return fundTypes

In [256]:
def getDfFromXl(excel):
    headers = list(excel.values)[0]
    content = list(excel.values)[1:]
    return pd.DataFrame(data=content,columns=headers)

In [257]:
def scrapeTable(tableInfo, indexVal, topRowToIgnore=0 ):
    holdingsRow = tableInfo.find_all('tr')[topRowToIgnore:]
    holdTable = []
    for row in range(len(holdingsRow)):
        holdTable.append(list(holdingsRow[row].stripped_strings))
    
    tableDf = pd.DataFrame(holdTable[1:], index=[[indexVal]*len(holdTable[1:])], columns=holdTable[:1][0])
    return tableDf

In [258]:
def returnFloat(contStr, toReplace=''):
    try:
        val = float(contStr.replace(toReplace,''))
    except ValueError:
        return contStr
    return val

In [259]:
def getValFromTable(column):
    columnVal = list(column.stripped_strings)[0]
    return columnVal

In [260]:
def scrapeSnapshot(fundCode):
    sShotResult = requests.get(vr_snapshot_path+fundCode)
    parsedSS = BeautifulSoup(sShotResult.content, 'lxml')
    snapContents = {}
    
    fundName = parsedSS.find('h1', class_='snapshot-fund-name')
    rating = parsedSS.find('span', itemprop = 'rating')
    snapContents['Fund Name'] = list(fundName.stripped_strings)[0]
    snapContents['Fund Code'] = int(fundCode)
    snapContents['Rating'] = int(0 if len(list(rating.stripped_strings)) is 0 else list(rating.stripped_strings)[0] )
    
    statusHtml = parsedSS.find('li', class_ = 'pull-right padding_right_none')
    statusString = list(statusHtml.stripped_strings)[0]
    status = ((statusString.split(':')[1]).split('|')[0]).strip()
    snapContents['Status'] = status
    
    if status.startswith('Closed for subscription'):
        return snapContents, True

    baseInfo = parsedSS.find('div', class_ = 'pull-left fundHeadRight')
    baseInfoStr = list(baseInfo.stripped_strings)
    snapContents['Category'] = baseInfoStr[1]
    snapContents['Assets'] = baseInfoStr[4].split('\r\n')[0]
    snapContents['Exp Ratio'] = baseInfoStr[6].split('%')[0]

    dateInfo = parsedSS.find('div', class_ = 'pull-left change-date')
    dateStr = list(dateInfo.stripped_strings)
    snapContents['Date'] = (dateStr[0].split('as on'))[1].strip()

    basicDetailsInfo = parsedSS.find('div', class_ = 'pull-left sectionHead margin_top15px basic-and-investment-details')
    basicDetailsList = list(basicDetailsInfo.stripped_strings)
    snapContents['Launch Date'] = basicDetailsList[4]
    snapContents['Benchmark'] = basicDetailsList[6]
    snapContents['Riskometer'] = basicDetailsList[8]
    snapContents['Risk Grade'] = basicDetailsList[10]
    snapContents['Return Grade'] = basicDetailsList[12]
    snapContents['T/O %'] = returnFloat(basicDetailsList[14],'%')
    snapContents['Type'] = basicDetailsList[16]
    snapContents['Ret sin Lau%'] = returnFloat(basicDetailsList[19],'%')
    snapContents['Mini Inv'] = returnFloat(basicDetailsList[23],',')
    snapContents['Min SIP'] = returnFloat(basicDetailsList[31],',')
    snapContents['Exit Load'] = basicDetailsList[43]
    
    return snapContents, False

In [261]:
def scrapePerformance(fundCode):
    performResult = requests.get(vr_performance_path+fundCode)
    parsedPerf = BeautifulSoup(performResult.content, 'lxml')
    
    perfContents = {}
    perfInfo = parsedPerf.find_all('div', class_ = 'pull-left sectionHead margin_top15px')

    riskMeasTable = perfInfo[3].find_all('tr')
    
    val = list(riskMeasTable[0].find('th').stripped_strings)[0]
    if val.startswith('Risk Measures'):  
        fundInfo = riskMeasTable[1].find_all('td')
        perfContents['Mean'] = returnFloat(getValFromTable(fundInfo[1]))
        perfContents['Std Dev'] =returnFloat(getValFromTable(fundInfo[2]))
        perfContents['Sharpe'] = returnFloat(getValFromTable(fundInfo[3]))
        perfContents['Sortino'] = returnFloat(getValFromTable(fundInfo[4]))
        perfContents['Beta'] = returnFloat(getValFromTable(fundInfo[5]))
        perfContents['Alpha'] = returnFloat(getValFromTable(fundInfo[6]))

        if len(riskMeasTable) > 4:
            fundRank = riskMeasTable[4].find_all('td')
            perfContents['Beta Rank'] = returnFloat(getValFromTable(fundRank[5]))
            perfContents['Alpha Rank'] = returnFloat(getValFromTable(fundRank[6]))

    trailRetTable=[]
    if val.startswith('Trailing Returns'):
        trailRetTable = riskMeasTable
    else:
        trailRetTable = perfInfo[4].find_all('tr')
        
    returnsInfo = trailRetTable[1].find_all('td')
    perfContents['YTD Ret'] = returnFloat(getValFromTable(returnsInfo[1]))
    perfContents['1W Ret'] = returnFloat(getValFromTable(returnsInfo[3]))
    perfContents['1M Ret'] = returnFloat(getValFromTable(returnsInfo[4]))
    perfContents['3M Ret'] = returnFloat(getValFromTable(returnsInfo[5]))
    perfContents['6M Ret'] = returnFloat(getValFromTable(returnsInfo[6]))
    perfContents['1Y Ret'] = returnFloat(getValFromTable(returnsInfo[7]))
    perfContents['3Y Ret'] = returnFloat(getValFromTable(returnsInfo[8]))

    if len(trailRetTable) > 4:
        returnsRank = trailRetTable[4].find_all('td')
        perfContents['YTD Rnk'] = returnFloat(getValFromTable(returnsRank[1]))
        perfContents['1W Rnk'] = returnFloat(getValFromTable(returnsRank[3]))
        perfContents['1M Rnk'] = returnFloat(getValFromTable(returnsRank[4]))
        perfContents['3M Rnk'] = returnFloat(getValFromTable(returnsRank[5]))
        perfContents['6M Rnk'] = returnFloat(getValFromTable(returnsRank[6]))
        perfContents['1Y Rnk'] = returnFloat(getValFromTable(returnsRank[7]))
        perfContents['3Y Rnk'] = returnFloat(getValFromTable(returnsRank[8]))

    return perfContents

In [262]:
def scrapePortfolio(fundCode):
    portfolioResult = requests.get(vr_portfolio_path+str(fundCode))
    parsedPortfol = BeautifulSoup(portfolioResult.content, 'lxml')
    
    pfContents = {}
    valInfo = parsedPortfol.find_all('td', class_ ='padding_top_none padding_bottom_none border_bottom_none')
    valList = list(valInfo[1].stripped_strings)

    pfContents['Num of Stks'] = returnFloat(valList[2])
    pfContents['Top 10 Stks%'] = returnFloat(valList[4])
    pfContents['Top 5 Stks%'] = returnFloat(valList[6])
    pfContents['Top 3 sect%'] = returnFloat(valList[8])
    pfContents['P/B Ratio'] = returnFloat(valList[10])
    pfContents['P/E Ratio'] = returnFloat(valList[12])

    aggInfo = parsedPortfol.find_all('td', class_ = 'align_right equity-fund-cell')
    if len(aggInfo)>0:
        pfContents['mkt cap(Cr)'] = returnFloat(getValFromTable(aggInfo[0]),',')
        pfContents['Giant(%)'] = returnFloat(getValFromTable(aggInfo[1]))
        pfContents['Large(%)'] = returnFloat(getValFromTable(aggInfo[2]))
        pfContents['Mid(%)'] = returnFloat(getValFromTable(aggInfo[3]))
        pfContents['Small(%)'] = returnFloat(getValFromTable(aggInfo[4]))
        pfContents['Tiny(%)'] = returnFloat(getValFromTable(aggInfo[5]))

    holdingsInfo = parsedPortfol.find_all('table', class_ ='fund-snapshot-port-holdings-equity')
    pf = scrapeTable(holdingsInfo[0],int(fundCode), 1)
    
    return pfContents, pf

In [263]:
def scrapeAnalysis(fundCode):
    analysisResult = requests.get(vr_analysis_path+str(fundCode))
    parsedAnalysis = BeautifulSoup(analysisResult.content, 'lxml')
    
    anContents = {}
    analysisInfo = parsedAnalysis.find_all('tr', class_='')
    anContents['Fund Manager'] = list(analysisInfo[2].stripped_strings)[1]
    
    return anContents

In [264]:
def scrapeEtPerf(fundCode):
    etPerfResult = requests.get(et_performance_path+str(fundCode)+'.cms')
    parsedEtPerf = BeautifulSoup(etPerfResult.content, 'lxml')
    
    etPerfContents = {}
    etPerfInfo = parsedEtPerf.find_all('div', class_='tabing')

    currentYrInfo = etPerfInfo[0].find_all('div', class_ = 'current')
    currentYrVals = list(currentYrInfo[0].stripped_strings)
    etPerfContents['YTD Q1'] = returnFloat(currentYrVals[1])
    etPerfContents['YTD Q2'] = returnFloat(currentYrVals[2])
    etPerfContents['YTD Q3'] = returnFloat(currentYrVals[3])
    etPerfContents['YTD Q4'] = returnFloat(currentYrVals[4])

    previousYrInfo = etPerfInfo[0].find_all('div', class_ = 'withbackground')
    
    if len(previousYrInfo)>0:
        oneYrVals = list(previousYrInfo[0].stripped_strings)
        etPerfContents['1Y Q1'] = returnFloat(oneYrVals[1])
        etPerfContents['1Y Q2'] = returnFloat(oneYrVals[2])
        etPerfContents['1Y Q3'] = returnFloat(oneYrVals[3])
        etPerfContents['1Y Q4'] = returnFloat(oneYrVals[4])

        if len(previousYrInfo)>1:
            twoYrVals = list(previousYrInfo[1].stripped_strings)
            etPerfContents['2Y Q1'] = returnFloat(twoYrVals[1])
            etPerfContents['2Y Q2'] = returnFloat(twoYrVals[2])
            etPerfContents['2Y Q3'] = returnFloat(twoYrVals[3])
            etPerfContents['2Y Q4'] = returnFloat(twoYrVals[4])

    etTechTerms = parsedEtPerf.find_all('div', class_='borDotted')
    etPerfContents['R.Squared'] = returnFloat(list(etTechTerms[3].stripped_strings)[3])
    etPerfContents['Treynor'] = returnFloat(list(etTechTerms[6].stripped_strings)[3])
    etPerfContents['Inf Ratio'] = returnFloat(list(etTechTerms[7].stripped_strings)[3])
    
    return etPerfContents

In [265]:
def scrapeEtPf(fundCode):
    etPfResult = requests.get(et_portfolio_path+str(fundCode)+'.cms')
    parsedEtPf = BeautifulSoup(etPfResult.content, 'lxml')
    
    etPfContents = {}
    etPfInfo = parsedEtPf.find_all('div', class_='pRow')
    etPfContents['Eqty(%)'] = returnFloat(list(etPfInfo[0].stripped_strings)[1].split('%')[0])
    etPfContents['Dbt(%)'] = returnFloat(list(etPfInfo[1].stripped_strings)[1].split('%')[0])
    etPfContents['Otr(%)'] = returnFloat(list(etPfInfo[2].stripped_strings)[1].split('%')[0])
    
    return etPfContents

In [266]:
def createFundLine(fundColumns, fundDict):
    line = []
    for col in fundColumns:
        try:
            colValue = fundDict[col]
        except KeyError:
            colValue = ''
        line.append(colValue)
    return line

In [267]:
print(str(datetime.now()))

fundTypes = readMasterExcel()

for sheet in list(fundTypes.keys()):
    print(sheet)
    if isSingleFundType is True:
        print('inside first')
        if singleFundType != sheet:
            print('inside second')
            continue
    try:
        load_workbook(details_file_path+sheet+date.today().strftime('_%d_%b_%Y')+'.xlsx')
        print(sheet + ' has already been read today!!')
        continue
    except FileNotFoundError:
        print('Redding type = '+sheet)
        pass
    
    sheetDf = getDfFromXl(fundTypes[sheet])
    print('starting processing '+ sheet)
    
    wb = Workbook()
    fundSheet = wb.active
    fundSheet.title = 'funds'
    pfSheet = wb.create_sheet(title='portfolio')
    
    fundSheet.append(list(sheetDf.columns))
        
    for fundCode in sheetDf['Fund Code'].values:
        fundDict = {}
        print('starting scrapping '+ str(fundCode))
        
        #fundDict.update(scrapeSnapshot(str(fundCode)))
        snFund, isStatusClosed = scrapeSnapshot(str(fundCode))
        fundDict.update(snFund)
        print('finished snapshot')
        
        if isStatusClosed is False:  
            fundDict.update(scrapePerformance(str(fundCode)))
            print('finished performance')
            fundDict.update(scrapeAnalysis(str(fundCode)))
            print('finished analysis')
            fundDict.update(scrapeEtPerf(str(fundCode)))
            print('finished ER performance')
            fundDict.update(scrapeEtPf(str(fundCode)))
            print('finished ET portfolio')
            
            if not sheet.startswith('global'):
                pfFund, pf = scrapePortfolio(str(fundCode))
                print('finished portfolio')
                fundDict.update(pfFund)
                for line in dataframe_to_rows(pf, index=False):
                    pfSheet.append(line)
                
        fundLine = createFundLine(sheetDf.columns, fundDict)
        print('created Line')
        fundSheet.append(fundLine)
    
    
    wb.save(details_file_path+sheet+date.today().strftime('_%d_%b_%Y')+'.xlsx')
    print(sheet+'.... has been saved')
    
print(str(datetime.now()))
    


2018-05-08 17:55:01.010959
hybrid equity
inside first
inside second
small cap
inside first
inside second
mid cap
inside first
inside second
elss
inside first
inside second
global
inside first
inside second
technology
inside first
inside second
multi cap
inside first
inside second
banking
inside first
inside second
infrastructure
inside first
inside second
large cap
inside first
Redding type = large cap
starting processing large cap
starting scrapping 17140
finished snapshot
finished performance
finished analysis
finished ER performance
finished ET portfolio
finished portfolio
created Line
starting scrapping 15883
finished snapshot
finished performance
finished analysis
finished ER performance
finished ET portfolio
finished portfolio
created Line
starting scrapping 16198
finished snapshot
finished performance
finished analysis
finished ER performance
finished ET portfolio
finished portfolio
created Line
starting scrapping 15831
finished snapshot
finished performance
finished analysis
fi